## Transfer learning to build an image recognition system that can identify pictures of dogs. 

## Build a Feature Extractor
The first step is to build a feature extractor that can extract training features from our images.

1. Load libraries, images, training data
2. Build a VGG model without the top layer (include_top=False)
3. Predict training data
4. Extract features and labels using joblib into a dat file

In [34]:
from PIL import Image
from pathlib import Path
import numpy as np
import joblib

from keras.preprocessing import image
from keras.applications import vgg16
from keras.models import model_from_json

# Training folder
training_folder = 'TrainingData/'
subfolder_dogs = '/dogs/'
subfolder_not_dogs = '/not_dogs/'

# Create path to folders with training data
dog_path = Path(training_folder) / subfolder_dogs
not_dog_path = Path(training_folder) / subfolder_not_dogs

# Create an images and labels list to store arrays; assign 0 or 1 as labels
images = []
labels = []

# Loop through not-dog images (not_dog_path)
for img in not_dog_path.glob('*.png'): # return a possibly-empty list of path names that match pathname "*.png"
    
    # Load the image from disk
    img = image.load_img(img)
    
    # Convert the image to a numpy array
    image_array = image.img_to_array(img) # keras helper function to convert image into an array
    
    # Add the image to the lsit of images
    images.append(image_array) # add image appray to list of images
    
    # Add labels which is 0 for not dogs
    labels.append(0) # add 0 to label array
    
# Load all the dog images
for img in dog_path.glob('*.png'):
    
    # Load the image from disk
    img = image.load_img(img)
    
    # Convert the image to a numpy array
    image_array = image.img_to_array(img)
    
    # Add the image to the list of images
    images.append(image_array)
    
    # For each dog image, the expected value should be 1
    labels.append(1)

### Create training data array (x & y)
Create an array called x_train that will have the training data. Keras expects all of our training images to be a numpy array instead of a normal Python list.

1. Create a single numpy array with all the images laoded
2. Convert the labels to a numpy array
3. Normalize from 0 to 1
4. Pre-trained neural network
5. Extract features for each image (all in one pass)
6. Save the array of extracted features to a file
7. Save the matching array of expected values to a file

In [ ]:
# Create a single numpy array with all the images
x_train = np.array(images)

# Convert labels to a numpy array as well
y_train = np.array(labels)

# Normalize
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to extract features; remove last layer of the neural network
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3)) 

# Extract features for each image
# Features x array will now contain the set of features that represent each of the training images in our dataset
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, 'x_train.dat')

# Save the matching array of expected values to a file
joblib.dump(y_train, 'y_train.dat') # These files contain the features and labels that represent our training data

## Training a New Neural Network w/ Extracted Features (Load)
We've used the pre trained neural network to extract features from our training images. Now we're ready to train a new neural network that uses those extracted features.

### Differences
The code is exactly like training any other neural network but with two small differences. 
1. The first difference is how we load our training data Instead of loading raw images to train with, we're gonna load the features that we extracted with the pre trained VGG 16 neural network. If you look at the file list on the left, you can see that we already have our extracted features stored in a file called x train.dat and our labels stored in a file called y train.dat.

2. The second difference is in how we define our layers. Since we use VGG 16 to extract features from our image, this neural network has no convolutional layers. Instead it only has the final dense layers of the neural network. These are the only layers that we'll be retraining.

In [ ]:
# Load data set
x_train = joblib.load('x_train.dat')
y_train = joblb.load('y_train.dat')

### Create a model and add layers
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=1,
    shuffle=True
)

# Save neural network structure
transfer_model_structure = model.to_json()
f = Path('transfer_model_structure.json')
f.write_text(transfer_model_structure)

# Save neural network's trained weights
model.save_weights('transfer_model_weights.h5')

## Make Predictions with Transfer Learning

We have used the pre-trained models, extracted features, created a new model that will train the dense layers, extracted the structure and trained weights using the combination of the two. Now we will make predictions on new data.

In [ ]:
# Load the json file that contains the transfer model structure
f = Path('transfer_model_structure.json')
model_structure = f.read_text()

# Recreate the Keras model object from the json daa
model = model_from_json(model_structure)

# Re-load the transfer model's trained weights
model.load_weights('transfer_model_weights.h5')

# Load an image file to test and resize it to 64x64
img = image.load_img('Images/23-potato.png', target_size=(64, 64))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image
images = np.expand_dims(image_array, axis=0)

# Normalize the data
images = vgg16.preprocess_input(images)

# Use the pre-trained neural network to extract features from the test image
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using the model
results = model.predict(features)

# Check first result since we only have one image
single_result = results[0][0]

# Print the result
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

Image.open('Images/23-potato.png')